# Paint Manufacturing Quality Crisis Analysis

## Business Problem
- **Before automation:** 99% pass rate
- **After automation:** 67% pass rate (33% failure rate!)
- **Mission:** Identify root causes and provide actionable recommendations

## Analysis Structure
Following the technical assessment requirements:
1. **Part 1:** Data Exploration & Understanding (45-60 min)
2. **Part 2:** Diagnostic Analysis (60-75 min)
3. **Part 3:** Predictive Modeling (45-60 min)
4. **Part 4:** Recommendations & Communication

---

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


# Part 1: Data Exploration & Understanding

**Objectives:**
- Understand data structure and quality issues
- Identify key variables and relationships
- Perform initial statistical analysis
- Formulate hypotheses about failure causes

In [2]:
# Load and examine the data
df = pd.read_csv('../data/paint_production_data.csv')

print("=== DATASET OVERVIEW ===")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"Date range: {df['Production_Date'].min()} to {df['Production_Date'].max()}")

print("\n=== DATA QUALITY ===")
print("Missing values:")
missing = df.isnull().sum()
for col, count in missing[missing > 0].items():
    pct = count / len(df) * 100
    print(f"  {col}: {count:,} ({pct:.1f}%)")

print(f"\nDuplicates: {df.duplicated().sum()}")

print("\n=== BUSINESS CONTEXT ===")
print(f"Total dosing events: {len(df):,}")
print(f"Unique batches: {df['Batch_ID'].nunique():,}")
print(f"Unique recipes: {df['Recipe_Name'].nunique()}")
print(f"Dosing stations: {df['Dosing_Station'].nunique()} ({sorted(df['Dosing_Station'].unique())})")
print(f"Events per batch (avg): {len(df) / df['Batch_ID'].nunique():.1f}")

df.head()

=== DATASET OVERVIEW ===
Shape: (89818, 10)
Columns: ['Batch_ID', 'Production_Date', 'Production_Time', 'Recipe_Name', 'Num_Ingredients', 'Dosing_Station', 'Target_Amount', 'Actual_Amount', 'Facility_Temperature', 'QC_Result']
Date range: 2024-01-01 to 2024-12-30

=== DATA QUALITY ===
Missing values:
  Actual_Amount: 1,797 (2.0%)
  Facility_Temperature: 898 (1.0%)

Duplicates: 5

=== BUSINESS CONTEXT ===
Total dosing events: 89,818
Unique batches: 6,500
Unique recipes: 48
Dosing stations: 7 (['D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07'])
Events per batch (avg): 13.8


,Batch_ID,Production_Date,Production_Time,Recipe_Name,Num_Ingredients,Dosing_Station,Target_Amount,Actual_Amount,Facility_Temperature,QC_Result
0,BATCH_000001,2024-11-01,22:28:11,RAL_5002,17,D02,5.961,6.085,26.9,failed
1,BATCH_000001,2024-11-01,22:28:11,RAL_5002,17,D03,37.510,39.810,26.9,failed
2,BATCH_000001,2024-11-01,22:28:11,RAL_5002,17,D01,21.491,21.115,26.9,failed
3,BATCH_000001,2024-11-01,22:28:11,RAL_5002,17,D07,4.225,4.276,26.9,failed
4,BATCH_000001,2024-11-01,22:28:11,RAL_5002,17,D03,1.918,2.005,26.9,failed


In [5]:
# Critical insight: QC results are at BATCH level, not event level
# We need to aggregate dosing events to batch level for proper analysis

print("=== QUALITY ANALYSIS ===")
event_failure_rate = (df['QC_Result'] == 'failed').mean()
print(f"Event-level failure rate: {event_failure_rate:.1%}")

# Batch-level failure rate (the real business metric)
batch_qc = df.groupby('Batch_ID')['QC_Result'].first()
batch_failure_rate = (batch_qc == 'failed').mean()
print(f"Batch-level failure rate: {batch_failure_rate:.1%} ← KEY BUSINESS METRIC")

print(f"\nDaily production: ~{df['Batch_ID'].nunique() / 365:.0f} batches/day")
print(f"Failed batches per day: ~{batch_failure_rate * df['Batch_ID'].nunique() / 365:.1f}")

=== QUALITY ANALYSIS ===
Event-level failure rate: 36.7%
Batch-level failure rate: 32.7% ← KEY BUSINESS METRIC

Daily production: ~18 batches/day
Failed batches per day: ~5.8


In [6]:
# Create batch-level dataset for analysis
print("=== CREATING BATCH-LEVEL DATASET ===")

# Calculate dosing errors
df['Dosing_Error'] = abs(df['Actual_Amount'] - df['Target_Amount'])

# Aggregate to batch level
batch_df = df.groupby('Batch_ID').agg({
    'Production_Date': 'first',
    'Recipe_Name': 'first',
    'Num_Ingredients': 'first',
    'QC_Result': 'first',
    'Facility_Temperature': 'mean',
    'Dosing_Error': ['mean', 'max', 'std'],
    'Target_Amount': 'sum',
    'Actual_Amount': 'sum',
    'Dosing_Station': 'nunique'
}).round(4)

# Flatten column names
batch_df.columns = ['_'.join(col).strip() if col[1] else col[0] for col in batch_df.columns]
batch_df = batch_df.reset_index()

# Create target variable (QC_Result becomes QC_Result_first after flattening)
batch_df['Failed'] = (batch_df['QC_Result_first'] == 'failed').astype(int)

print(f"Batch dataset shape: {batch_df.shape}")
print(f"Batch failure rate: {batch_df['Failed'].mean():.1%}")
print("\nBatch dataset ready for analysis")
batch_df.head()

=== CREATING BATCH-LEVEL DATASET ===
Batch dataset shape: (6500, 13)
Batch failure rate: 32.7%

Batch dataset ready for analysis


,Batch_ID,Production_Date_first,Recipe_Name_first,Num_Ingredients_first,QC_Result_first,Facility_Temperature_mean,Dosing_Error_mean,Dosing_Error_max,Dosing_Error_std,Target_Amount_sum,Actual_Amount_sum,Dosing_Station_nunique,Failed
0,BATCH_000001,2024-11-01,RAL_5002,17,failed,28.2588,0.2356,2.300,0.5572,102.860,106.024,7,1
1,BATCH_000002,2024-02-02,NCS_S2030-B40G,10,passed,20.6000,0.1319,0.806,0.2443,85.528,85.899,3,0
2,BATCH_000003,2024-02-08,RAL_8002,9,passed,22.0000,0.5047,3.386,1.1012,68.896,73.302,4,0
3,BATCH_000004,2024-01-27,RAL_7035,11,passed,20.8000,0.1422,0.771,0.2508,57.895,59.263,4,0
4,BATCH_000005,2024-09-18,RAL_1007,6,passed,24.6000,0.6682,1.935,0.8328,129.296,129.435,4,0


In [7]:
# Initial hypothesis testing
print("=== INITIAL HYPOTHESIS TESTING ===")

# Hypothesis 1: Recipe complexity affects failure rate
print("\n1. RECIPE COMPLEXITY HYPOTHESIS:")
complexity_analysis = batch_df.groupby('Num_Ingredients_first')['Failed'].agg(['count', 'mean']).round(3)
complexity_analysis.columns = ['Batch_Count', 'Failure_Rate']
print(complexity_analysis.head(10))

# Test threshold at 15 ingredients
simple = batch_df[batch_df['Num_Ingredients_first'] <= 15]
complex_recipes = batch_df[batch_df['Num_Ingredients_first'] > 15]
print(f"\nSimple recipes (≤15 ingredients): {simple['Failed'].mean():.1%} failure rate")
print(f"Complex recipes (>15 ingredients): {complex_recipes['Failed'].mean():.1%} failure rate")
print(f"Difference: {complex_recipes['Failed'].mean() - simple['Failed'].mean():.1%}")

# Hypothesis 2: Temperature affects failure rate
print("\n2. TEMPERATURE HYPOTHESIS:")
temp_data = batch_df.dropna(subset=['Facility_Temperature_mean'])
print(f"Temperature range: {temp_data['Facility_Temperature_mean'].min():.1f}°C to {temp_data['Facility_Temperature_mean'].max():.1f}°C")

# Test optimal range 20-25°C
optimal_temp = temp_data[(temp_data['Facility_Temperature_mean'] >= 20) & (temp_data['Facility_Temperature_mean'] <= 25)]
suboptimal_temp = temp_data[(temp_data['Facility_Temperature_mean'] < 20) | (temp_data['Facility_Temperature_mean'] > 25)]
print(f"Optimal temp (20-25°C): {optimal_temp['Failed'].mean():.1%} failure rate")
print(f"Suboptimal temp: {suboptimal_temp['Failed'].mean():.1%} failure rate")
print(f"Difference: {suboptimal_temp['Failed'].mean() - optimal_temp['Failed'].mean():.1%}")

# Hypothesis 3: Station performance varies
print("\n3. STATION PERFORMANCE HYPOTHESIS:")
station_performance = df.groupby('Dosing_Station')['QC_Result'].apply(lambda x: (x == 'failed').mean()).sort_values(ascending=False)
print("Station failure rates:")
for station, rate in station_performance.items():
    print(f"  {station}: {rate:.1%}")

print(f"\nStation performance spread: {station_performance.max() - station_performance.min():.1%}")

=== INITIAL HYPOTHESIS TESTING ===

1. RECIPE COMPLEXITY HYPOTHESIS:
                       Batch_Count  Failure_Rate
Num_Ingredients_first                           
5                              812         0.267
6                              218         0.275
7                              346         0.309
8                              258         0.302
9                              730         0.284
10                             410         0.298
11                             405         0.269
12                             413         0.300
13                             171         0.298
14                             394         0.284

Simple recipes (≤15 ingredients): 28.9% failure rate
Complex recipes (>15 ingredients): 41.6% failure rate
Difference: 12.7%

2. TEMPERATURE HYPOTHESIS:
Temperature range: 15.3°C to 32.1°C
Optimal temp (20-25°C): 28.9% failure rate
Suboptimal temp: 39.5% failure rate
Difference: 10.7%

3. STATION PERFORMANCE HYPOTHESIS:
Station failure rate

## Part 1 Summary

**Key Findings:**
- **Batch failure rate: 32.7%** (the critical business metric)
- **Recipe complexity effect:** Simple (≤15 ingredients) = 28.9% vs Complex (>15) = 41.6%
- **Temperature effect:** Optimal (20-25°C) = 28.9% vs Suboptimal = 39.5%
- **Station variation:** Performance spread of ~5% between best and worst stations

**Hypotheses for Part 2:**
1. Recipe complexity >15 ingredients significantly increases failure risk
2. Temperature outside 20-25°C range increases failure risk
3. Certain dosing stations have systematic performance issues
4. Multiple factors may interact (multiplicative effects)

**Data Quality:** Manageable missing values (2,695 total), realistic industrial dataset

---
**Part 1 Status: ✅ COMPLETE**